In [17]:
import numpy as np
import pandas as pd
import networkx as nx 
import random
import matplotlib.pyplot as plt

np.random.seed(1)
random.seed(1)

Functions that will be used:


In [18]:
def one_iteration(G, gamma, beta) : 
    list_healthy = []
    list_infected = []
    # Maybe there is abetter way to do that
    for node, attribute in nx.get_node_attributes(G, "Status").items() : 
        if attribute == "Infected" : 
            list_infected.append(node)
        else : list_healthy.append(node)
    
    # CREATE A LIST WITH THE INFECTED NODES THAT WILL RECOVER IN THIS STEP
    list_recovered = []
    for i in list_infected : 
        if random.uniform(0,1) < gamma : 
            list_recovered.append(i)

    # CREATE A LIST WITH THE HEALTHY NODES THAT WILL BE INFECTED IN THIS STEP
    for i in list_healthy : 
        neighbours_infected = [v for v in nx.neighbors(G, i) if G.nodes[v]["Status"] == "Infected"]
        if not (random.uniform(0,1) < pow((1-beta),len(neighbours_infected))) :
            G.nodes[i]["Status"] = "Infected" # (instead of creating a list we directly change their status)
    
    # UPDATE STATUS FOR THE NODES RECOVERED
    for i in list_recovered : 
        G.nodes[i]["Status"] = "Susceptible"

    # GET THE LISTS OF INFECTED AND SUSCEPTIBLE PEOPLE
    list_infected = [u for (u,d) in nx.get_node_attributes(G, "Status").items() if d == 'Infected']
    list_healthy = [u for (u,d) in nx.get_node_attributes(G, "Status").items() if d == 'Susceptible']
    return G # , list_infected, list_healthy

We set the parameters of the model and create the netwroks:

In [19]:
# Parameters
total_pop = 1000
p_0 = 0.01
gamma = 0.1
beta = 0.3
t_max = 500

# Networks to be studied 
G_Kn = nx.complete_graph(total_pop)
G_BA = nx.barabasi_albert_graph(total_pop, m = 5, seed = 1)
G_WS = nx.watts_strogatz_graph(total_pop, k = 4, p = 0.2, seed = 1)
# G_tree = nx.tree_graph(total_pop, seed = 1)
# G_star = nx.star_graph(total_pop, seed = 1)
# G_lattice = nx.latt
G_ER = nx.erdos_renyi_graph(total_pop, p = 0.5, seed = 1) # SHOULD CHANGE P TO SOMETHING THAT IS LESS CONNECTED. 




Set the status of each node at time 0

In [20]:
# At first all the population is helathy
nx.set_node_attributes(G_Kn, "Susceptible", "Status")
nx.set_node_attributes(G_BA, "Susceptible", "Status")
nx.set_node_attributes(G_WS, "Susceptible", "Status")
nx.set_node_attributes(G_ER, "Susceptible", "Status")

# Choose the nodes that will be infected at the begining
infected_0 = np.random.choice(total_pop, int(p_0*total_pop), replace=False)

# infected_0
# G_Kn.nodes(data =True)
# G_BA.nodes(data =True)
# G_WS.nodes(data =True)
# G_ER.nodes(data =True)

# Change the status of those nodes to infected
for i in infected_0 : 
    G_Kn.nodes[i]["Status"] = "Infected"
    G_BA.nodes[i]["Status"] = "Infected"
    G_WS.nodes[i]["Status"] = "Infected"
    G_ER.nodes[i]["Status"] = "Infected"


# In case we want to check it works
# infected_neightbour_i = len([v for v in nx.neighbors(G_Kn, 0) if G_Kn.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_BA, 0) if G_BA.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_WS, 0) if G_WS.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i = len([v for v in nx.neighbors(G_ER, 0) if G_ER.nodes[v]["Status"] == "Infected"])
# infected_neightbour_i

Code that computes the epidemic:

In [21]:
# Create DataFrames and save situation at time 0
df_Kn = pd.DataFrame()
df_BA = pd.DataFrame()
df_WS = pd.DataFrame()
df_ER = pd.DataFrame()
df_Kn[0] = nx.get_node_attributes(G_Kn,"Status").values()
df_BA[0] = nx.get_node_attributes(G_BA,"Status").values()
df_WS[0] = nx.get_node_attributes(G_WS,"Status").values()
df_ER[0] = nx.get_node_attributes(G_ER,"Status").values()


# Iterate trought time 
for i in range(t_max) : # We will stop at t_max = 500 which should be enough considering our networks are of the size of 1000 nodes 
    # Simulates one step
    G_Kn = one_iteration(G_Kn, gamma, beta)
    G_BA = one_iteration(G_BA, gamma, beta)
    G_WS = one_iteration(G_WS, gamma, beta)
    G_ER = one_iteration(G_ER, gamma, beta)
    # Save the results of the step in the DataFrames
    df_Kn[i+1] = nx.get_node_attributes(G_Kn,"Status").values()
    df_BA[i+1] = nx.get_node_attributes(G_BA,"Status").values()
    df_WS[i+1] = nx.get_node_attributes(G_WS,"Status").values()
    df_ER[i+1] = nx.get_node_attributes(G_ER,"Status").values()

df_Kn.to_csv("Simulation_Kn.csv", index = False)
df_BA.to_csv("Simulation_BA.csv", index = False)
df_WS.to_csv("Simulation_WS.csv", index = False)
df_ER.to_csv("Simulation_ER.csv", index = False)


C:\Users\lluis\AppData\Local\Temp\ipykernel_15768\1093077408.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_Kn[i+1] = nx.get_node_attributes(G_Kn,"Status").values()
C:\Users\lluis\AppData\Local\Temp\ipykernel_15768\1093077408.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_BA[i+1] = nx.get_node_attributes(G_BA,"Status").values()
C:\Users\lluis\AppData\Local\Temp\ipykernel_15768\1093077408.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [22]:
# Modify data to be shown
# df_Kn.replace(to_replace="Susceptible", value = 0, inplace = True)
# df_Kn.replace(to_replace="Infected", value = 1, inplace = True)
# df_Kn.replace(to_replace=0, value = "Susceptible", inplace = True)
# df_Kn.replace(to_replace=1, value = "Infected", inplace = True)

# Read files in case is not computed previously
df_Kn = pd.read_csv("Simulation_Kn.csv")
df_BA = pd.read_csv("Simulation_BA.csv")
df_WS = pd.read_csv("Simulation_WS.csv")
df_ER = pd.read_csv("Simulation_ER.csv")


df_plot_Kn = df_Kn.apply(pd.Series.value_counts).transpose() 
df_plot_BA = df_BA.apply(pd.Series.value_counts).transpose()
df_plot_WS = df_WS.apply(pd.Series.value_counts).transpose()
df_plot_ER = df_ER.apply(pd.Series.value_counts).transpose()


In [23]:
df_plot = pd.concat([df_plot_Kn, df_plot_BA, df_plot_WS, df_plot_ER], axis = 1)
df_plot.drop(df_plot.columns[[1, 3, 5, 7]], axis=1, inplace=True)
df_plot.reset_index(inplace=True)
df_plot.set_axis(['Time', 'Kn', 'BA', 'WS', 'ER'], axis='columns', inplace=True)

In [24]:
df_plot

,Time,Kn,BA,WS,ER
0,0,10,10,10,10
1,1,997,345,21,996
2,2,904,879,51,904
3,3,900,870,92,929
4,4,898,904,147,908
...,...,...,...,...,...
496,496,907,908,876,914
497,497,891,914,857,900
498,498,914,886,868,918
499,499,900,906,877,910
